In [288]:
import configparser
from pathlib import Path
import pandas as pd
import pandas.io.sql as pandasql
from psycopg2.extras import execute_values
from psycopg2 import connect
import numpy
import matplotlib as mpl
from IPython.display import HTML, display
import rick
import numpy as np
import matplotlib.pyplot as plt
from setuptools import setup, find_packages
CONFIG = configparser.ConfigParser()
CONFIG.read(str(Path.home().joinpath('db.cfg')))
dbset = CONFIG['DBSETTINGS']
con = connect(**dbset)

In [332]:
merge_set = pd.read_sql('''select rank, start_vid, end_vid, length, link_set, new_start_vid, new_end_vid, new_length, new_link_set, st_hausdorffdistance  from test_route_ranks
                          ''',con)

In [346]:
link_used = []
new_sets = []
rows = []
unique_set = merge_set[['start_vid', 'end_vid']].drop_duplicates()

# loop through unique sets of intersection
# !!this will be ordered by priority and length eventually!!
for index, key in unique_set.reset_index(drop=True).iterrows(): 
    
    start_vid = unique_set['start_vid'].iloc[index]
    end_vid = unique_set['end_vid'].iloc[index]
    print(start_vid, end_vid)
    prepare_set = merge_set[(merge_set['start_vid'] == start_vid) & (merge_set['end_vid'] == end_vid)]
    link_set = prepare_set['link_set'].iloc[0]
    
    # Check if this link_set was used to match another set before
    # if not, selection best and merge
    if (link_set in link_used) == False:
        print('not used')
        # if there are more than one rank and length is less than 100 or something 
        
        if prepare_set['length'].iloc[0] < 100:    
            # if there are two possibility for this intersection set
            # then compare which one is better

            if prepare_set['rank'].shape[0] == 2: 
                rank1=prepare_set[(merge_set['rank']==1)]
                rank2=prepare_set[(merge_set['rank']==2)]

                # find hausdorffdistance difference and length difference
                hausorffdiff = abs(rank1['st_hausdorffdistance'].iloc[0]/rank2['st_hausdorffdistance'].iloc[0])
                lengthdiff = rank1['new_length'].iloc[0] - rank2['new_length'].iloc[0]

                # make selection based on hausdorffdistance difference and length difference
                # could try and play with the params a little bit
                if hausorffdiff < 5 and lengthdiff > 20:
                    select_set = 2
                else:
                    select_set = 1

                selection = merge_set[(merge_set['rank']==select_set)&(merge_set['start_vid']==start_vid)&(merge_set['end_vid']==end_vid)]
                new_set = np.array([selection['new_start_vid'].iloc[0], selection['new_end_vid'].iloc[0], selection['new_length'].iloc[0], selection['new_link_set'].iloc[0]])

                
                # find links that were used to merge, append it to another list
                # also have to find out if it was using a set of link that was used before
                old_links = selection['link_set'].iloc[0]
                new_links = selection['new_link_set'].iloc[0]
                additional_links = list(set(new_links).difference(old_links))
                
                if (additional_links in link_used) == False:
                    link_used.append(additional_links)
                    new_sets.append(new_set)
                    
            # if there is only one possibility
            # see if that possibility is viable 
            elif prepare_set['rank'].shape[0] == 1:
                selection = merge_set[(merge_set['rank']==1)&(merge_set['start_vid']==start_vid)&(merge_set['end_vid']==end_vid)]
                old_links = selection['link_set'].iloc[0]
                new_links = selection['new_link_set'].iloc[0]
                additional_links = list(set(new_links).difference(old_links))
                new_set = np.array([selection['new_start_vid'].iloc[0], selection['new_end_vid'].iloc[0], selection['new_length'].iloc[0], selection['new_link_set'].iloc[0]])
                if (additional_links in link_used) == False:
                    link_used.append(old_links)
                    link_used.append(additional_links)
                    new_sets.append(new_set)
                else:
                    print('was used before')
                
        
        # if the set length is longer than 100 then 
        # check to see if it was used to merge 
        # if yes dont append, if not append
        else: 
            print('longer than 100m')
            if prepare_set['link_set'].iloc[0] in link_used == True:
                print('was used before')
            else:
                new_set = np.array([prepare_set['start_vid'].iloc[0], prepare_set['end_vid'].iloc[0], prepare_set['length'].iloc[0], prepare_set['link_set'].iloc[0]])
                new_sets.append(new_set)
                link_used.append(prepare_set['link_set'].iloc[0])
                
    # if link_set as used, continue and dont append
    elif (link_used in link_set) == True:
        print('used')

for i in range(len(new_sets)):
    item = new_sets[i]
    length = item[2].astype(float)

    row = (item[0].astype(float), item[1].astype(float), length, item[3])
    #print(type(item[0].astype(int)),type(item[1]),type(item[2]),type(item[3]) )
    rows.append(row)
        
sql = '''insert into congestion.test_shortseg(start_vid, end_vid, length, link_set) VALUES %s'''    
with con:
    with con.cursor() as cur:
        execute_values(cur, sql, rows)    

        

30362987 30362990
[9456288911, 9456288901, 295895041]
[]
False
not used
link_used []
longer than 100m
30362987 862292856
[7980069200]
[[9456288911, 9456288901, 295895041]]
False
not used
link_used [[9456288911, 9456288901, 295895041]]
old_links[7980069200]
new_links[7980069200, 7980069210]
additional_links[7980069210]
30362990 30362979
[7546081061, 10558928811, 10708227301, 10708227291]
[[9456288911, 9456288901, 295895041], [7980069210]]
False
not used
link_used [[9456288911, 9456288901, 295895041], [7980069210]]
longer than 100m
30362990 30362987
[295895040, 9456288900, 9456288910]
[[9456288911, 9456288901, 295895041], [7980069210], [7546081061, 10558928811, 10708227301, 10708227291]]
False
not used
link_used [[9456288911, 9456288901, 295895041], [7980069210], [7546081061, 10558928811, 10708227301, 10708227291]]
longer than 100m
30362990 30414528
[295895030, 295894770, 10566913110, 10566913120, 8266390160, 8266390170, 9455932580, 9455932590, 7994232670, 7994232680]
[[9456288911, 94562

/etc/jupyterhub/.venv/lib/python3.5/site-packages/ipykernel_launcher.py:30: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

/etc/jupyterhub/.venv/lib/python3.5/site-packages/ipykernel_launcher.py:31: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.




longer than 100m
30414551 30414540
[295896641]
[[9456288911, 9456288901, 295895041], [7980069210], [7546081061, 10558928811, 10708227301, 10708227291], [295895040, 9456288900, 9456288910], [295895030, 295894770, 10566913110, 10566913120, 8266390160, 8266390170, 9455932580, 9455932590, 7994232670, 7994232680], [295893130, 295892440, 295892050, 295891530, 295891250, 12056277700, 12056277710, 7545625650, 9488531850, 9488531860, 7545625620, 10558927980, 10558927990, 9456275600], [7980069240, 7980069250, 295892560, 10566913020, 10566913030, 10566912930, 10566912940, 11590148400, 11590148410, 295891890, 295891870, 295891700], [7980069201], [8266391400, 9456317170, 9456317180, 12056282460, 12056282470, 295880830], [11050776541, 11050776531, 9258634461, 11629851141, 11629851131, 11053320161, 11053320151], [7980093400, 7980093410], [295880831, 12056282471, 12056282461, 9456317181, 9456317171, 8266391401], [295879450, 9456317150, 9456317160, 9961041330, 12056278910, 12056278920, 9960023910, 996

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

